In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


In [2]:
#Check the SparkContext version
sc.version

u'2.3.0'

In [3]:
# Reading dfZika file generated from the cleaning part of the assignment

dfZika = sc.textFile("dfZikaNew.csv") 

print(dfZika)
dfZika.first()

dfZikaNew.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


u'19/03/2016;Argentina-Buenos_Aires;province;cumulative_confirmed_local_cases;AR0001;0;cases;Argentina'

In [4]:
#Check the number of partitions
dfZika.getNumPartitions() 

2

In [5]:
# Get records data fields (string split separate by ;
dfZika_fields = dfZika.map(lambda line: line.split(";"))
#Taking 5 records
print(dfZika_fields.take(5))
#Showing the firs Record
print("dfZika record data: " +format(dfZika_fields.first()))

[[u'19/03/2016', u'Argentina-Buenos_Aires', u'province', u'cumulative_confirmed_local_cases', u'AR0001', u'0', u'cases', u'Argentina'], [u'19/03/2016', u'Argentina-Buenos_Aires', u'province', u'cumulative_probable_local_cases', u'AR0002', u'0', u'cases', u'Argentina'], [u'19/03/2016', u'Argentina-Buenos_Aires', u'province', u'cumulative_confirmed_imported_cases', u'AR0003', u'2', u'cases', u'Argentina'], [u'19/03/2016', u'Argentina-Buenos_Aires', u'province', u'cumulative_probable_imported_cases', u'AR0004', u'1', u'cases', u'Argentina'], [u'19/03/2016', u'Argentina-Buenos_Aires', u'province', u'cumulative_cases_under_study', u'AR0005', u'127', u'cases', u'Argentina']]
dfZika record data: [u'19/03/2016', u'Argentina-Buenos_Aires', u'province', u'cumulative_confirmed_local_cases', u'AR0001', u'0', u'cases', u'Argentina']


In [6]:
# Gettings Statistics from the value
print("Statistics ->", dfZika_fields.map(lambda x: int(x[5])).stats())

('Statistics ->', (count: 107478, mean: 48.6345484657, stdev: 1343.198120820932, max: 161241.0, min: 0.0))


In [7]:
#Getting customized statistics 

# Count the number of cases
total_cases = dfZika.count()
total_confirmed_cases = dfZika_fields.map(lambda fields: fields[3]).filter(lambda line: "confirmed" in line).count()

# Count the number of distinct countries
num_countries=dfZika_fields.map(lambda line: line[7]).distinct().count()

# Count the number of diferent cases 
num_cases = dfZika_fields.map(lambda fields: fields[3]).distinct().count()
print ("Some Statistics -> Total Cases: %d, Total Confirmed Cases: %d, Countries: %d, Distinct cases: %d" % 
(total_cases,total_confirmed_cases, num_countries, num_cases))


Some Statistics -> Total Cases: 107478, Total Confirmed Cases: 47469, Countries: 14, Distinct cases: 148


In [8]:
#Listing the 148 cases 
print(dfZika_fields.map(lambda fields: fields[3]).distinct().collect())

[u'total_zika_confirmed', u'confirmed_eyepain', u'zika_confirmed_4weeks', u'cumulative_probable_imported_cases', u'GBS_reported_cumulative_2015-2016', u'microcephaly_not', u'weekly_Zika_confirmed_pending', u'confirmed_rash', u'confirmed_age_60_plus', u'cumulative_probable_local_cases', u'microcephaly_under_investigation', u'municipality_microcephaly', u'total_zika_confirmed_ages_20-49yrs_M', u'total_zika_suspected_F', u'total_zika_confirmed_Not-Aplicable', u'weekly_zika_confirmed', u'cumulative_suspected_age_5-9', u'cumulative_suspected_age_60_plus', u'yearly_cumulative_male', u'confirmed_age_over59', u'total_zika_confirmed_ages_10-14yrs_F', u'confirmed_female', u'confirmed_age_20-24', u'total_zika_confirmed_imported', u'zika_confirmed_pcr_cumulative', u'confirmed_age_25-34', u'total_zika_confirmed_pregnant_cumulative', u'total_zika_suspected_cumulative', u'gbs_reported', u'total_zika_confirmed_M', u'arbovirus_suspected_cumulative_2015', u'zika_reported_local', u'total_zika_confirmed_a

In [9]:
#Using .map .. distinct .sortBy .collect to get the list of countries to Show the countries, 
#in this case I'm using United_States_Virgin_Islands as a country or let's say special location

print(dfZika_fields.map(lambda line: line[7]).distinct().sortBy(lambda x: x).collect())

[u'Argentina', u'Brazil', u'Colombia', u'Dominican_Republic', u'Ecuador', u'El_Salvador', u'Guatemala', u'Haiti', u'Mexico', u'Nicaragua', u'Panama', u'Puerto_Rico', u'United_States', u'United_States_Virgin_Islands']


In [10]:
#Getting the number of reports by country 
dfZika_fields.map(lambda line: line[7])\
.map(lambda word: (word, 1))\
.groupByKey()\
.mapValues(lambda values: sum(values))\
.sortBy(lambda x: x)\
.collect()

[(u'Argentina', 2016),
 (u'Brazil', 4167),
 (u'Colombia', 86887),
 (u'Dominican_Republic', 5716),
 (u'Ecuador', 796),
 (u'El_Salvador', 990),
 (u'Guatemala', 479),
 (u'Haiti', 52),
 (u'Mexico', 2880),
 (u'Nicaragua', 125),
 (u'Panama', 148),
 (u'Puerto_Rico', 260),
 (u'United_States', 2453),
 (u'United_States_Virgin_Islands', 509)]

In [11]:
# Getting the top 10  cases

print(dfZika_fields.map(lambda line: line[3])\
.map(lambda word: (word, 1))\
.groupByKey()\
.mapValues(lambda values: sum(values))\
.sortBy(lambda x: x)\
.top(10))


[(u'zika_suspected_pregnant_cumulative', 380), (u'zika_suspected_pregnant', 378), (u'zika_suspected_cumulative', 380), (u'zika_suspected_clinic', 16169), (u'zika_suspected_4weeks', 374), (u'zika_suspected', 28963), (u'zika_reported_travel', 709), (u'zika_reported_local', 709), (u'zika_reported', 311), (u'zika_pending', 20)]


In [12]:
#Getting the number of reports by type of location
dfZika_fields.map(lambda line: line[2])\
.map(lambda word: (word, 1))\
.groupByKey()\
.mapValues(lambda values: sum(values))\
.sortBy(lambda x: x)\
.collect()

[(u'Department', 95),
 (u'city', 14),
 (u'country', 856),
 (u'county', 1613),
 (u'department', 600),
 (u'district', 10),
 (u'municipality', 87554),
 (u'province', 7583),
 (u'region', 35),
 (u'state', 8341),
 (u'territory', 777)]

In [13]:
#Getting the confirmed cases by ranges of ages
confirmed_age_under_1 = dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_under_1")).collect()
confirmed_age_1to4= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_1-4")).collect()
confirmed_age_5to9= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_5-9")).collect()
confirmed_age_10to14= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_10-14")).collect()
confirmed_age_15to19= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_15-19")).collect()
confirmed_age_20to24= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_20-24")).collect()
confirmed_age_25to34= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_25-34" )).collect()
confirmed_age_35to49= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_35-49")).collect()
confirmed_age_50to59= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_50-59")).collect()
confirmed_age_60to64= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_60-64")).collect()
confirmed_age_60_plus= dfZika_fields.filter(lambda rec: (rec[3] == "confirmed_age_60_plus")).collect()

print (confirmed_age_under_1)
print (confirmed_age_1to4)
print (confirmed_age_5to9)
print (confirmed_age_10to14)
print (confirmed_age_15to19)
print (confirmed_age_20to24)
print (confirmed_age_25to34)
print (confirmed_age_35to49)
print (confirmed_age_50to59)
print ( confirmed_age_60to64)
print (confirmed_age_60_plus)



[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_under_1', u'PA0005', u'6', u'cases', u'Panama']]
[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_1-4', u'PA0006', u'10', u'cases', u'Panama']]
[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_5-9', u'PA0007', u'8', u'cases', u'Panama']]
[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_10-14', u'PA0008', u'15', u'cases', u'Panama']]
[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_15-19', u'PA0009', u'6', u'cases', u'Panama']]
[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_20-24', u'PA0010', u'18', u'cases', u'Panama']]
[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_25-34', u'PA0011', u'38', u'cases', u'Panama']]
[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_35-49', u'PA0012', u'50', u'cases', u'Panama']]
[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_50-59', u'PA0013', u'20', u'cases', u'Panama']]
[[u'11/04/2016', u'Panama', u'country', u'confirmed_age_60-64', u'PA0

In [14]:
#Check the recently report date

print(dfZika_fields.map(lambda fields: fields[0]).max())

30/05/2016


In [15]:
#Checking the oldest report date
print(dfZika_fields.map(lambda fields: fields[0]).min())

01/03/2016


In [16]:
#Checking the busy reporting dates
print(dfZika_fields.map(lambda fields: fields[0]).distinct().collect())

[u'17/05/2016', u'04/05/2016', u'14/06/2016', u'19/01/2016', u'09/02/2016', u'23/12/2015', u'16/02/2016', u'07/06/2016', u'02/01/2016', u'27/02/2016', u'19/05/2016', u'28/11/2015', u'25/06/2016', u'11/05/2016', u'09/06/2016', u'24/05/2016', u'08/03/2016', u'01/06/2016', u'22/03/2016', u'23/04/2016', u'09/04/2016', u'28/03/2016', u'27/04/2016', u'16/06/2016', u'27/12/2015', u'03/02/2016', u'11/03/2016', u'05/12/2015', u'22/05/2016', u'23/06/2016', u'21/06/2016', u'12/12/2015', u'09/12/2015', u'26/05/2016', u'16/12/2015', u'05/04/2016', u'23/02/2016', u'12/02/2016', u'29/12/2015', u'18/04/2016', u'29/02/2016', u'28/05/2016', u'15/03/2016', u'29/06/2016', u'25/04/2016', u'03/06/2016', u'17/03/2016', u'18/06/2016', u'29/04/2016', u'16/04/2016', u'30/04/2016', u'02/07/2016', u'18/02/2016', u'19/03/2016', u'12/04/2016', u'26/01/2016', u'10/02/2016', u'26/03/2016', u'16/01/2016', u'08/06/2016', u'20/02/2016', u'09/01/2016', u'13/06/2016', u'25/03/2016', u'05/05/2016', u'10/05/2016', u'14/03/2

In [17]:
# import matplotlib.pyplot as plt
# Zika_cases= dfZika_fields.map(lambda x: int(x[5])).collect()
# Zika_stats = dfZika_fields.map(lambda x: int(x[5])).stats()


# plt.hist(Zika_cases, bins=int((Zika_stats.max()-Zika_stats.min())+1), color='lightblue', density=True)
# fig = plt.gcf()
# fig.set_size_inches(26, 10)